DANIELA GARCÍA MILLÁN

In [1]:
!pip install -q  einops accelerate bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [3]:
import torch # PyTorch
import numpy as np
import getpass
import os
import transformers
import accelerate

import warnings
warnings.filterwarnings('ignore')

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

4.51.3
2.6.0+cu124
1.5.2


In [5]:
device

'cuda:0'

In [6]:
torch.random.manual_seed(42)

In [12]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [13]:
#id_model = "microsoft/Phi-3-mini-4k-instruct"  # no funciona

id_model = "google/gemma-2b"

#Caracteristicas del modelo

son adecuados para una variedad de tareas de generación de texto, que incluyen Respuesta a preguntas, resumen y razonamiento.
El modelo 27B se entrenó con 13 billones de tokens, el modelo 9B fue entrenado con 8 billones de tokens, y el modelo 2B se entrenó con 2 billones de tokens.
Se compone de una amplia gama de estilos lingüísticos, temas y vocabulario

Al exponer elmodelo, ayuda a aprender la sintaxis y los patrones de lenjuage para mejorar su capacidad a la hora de generar código o comprender preguntas.
AAdemas, en las matemáticas ayuda al modelo a aprender lógica el razonamiento, la representación simbólica.

Se han procesado filtrado de CSAM (material de abuso sexual infantil) como preparación de datos para garantizar la Exclusión de contenidos nocivos e ilegales.
 También filtra datos personales e informacion con datos sensibles

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map="auto",  # Mejor "auto" para que distribuya bien en GPUs disponibles
    torch_dtype="auto",  # Está bien
    trust_remote_code=True,  # Necesario a veces para modelos recientes
    attn_implementation="eager"  # Correcto, "eager" funciona bien si no quieres optimizaciones flash-attention
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

#Creación del Pipeline

In [19]:
generacion = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [20]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # from 0.1 to 0.9
    "do_sample": True,
}

In [21]:
#prompt = "¿Qué es la inteligencia artificial general?"
prompt = "¿Como se hace una tortilla española con pocas palabras?"
#prompt= "Erase una vez un instituto de educacion secundaria donde "
output = generacion(prompt, **generation_args)

In [22]:
output

[{'generated_text': '\n\nIn each of the following sentences, underline the correct form of the adjective or adverb in parentheses.\n\n<strong>Example 1</strong>. This fireworks display is (impressiver, $\\underline{{more~ impressive}}$) than last year’s was.\n\nIn high school we have <em>(more, manier)</em> classes that last longer than an hour.\n\nWrite the letter for the word that best completes given sentence.\n____________ The insurgent __________ seized political power from the elected leaders.\na. caste\nb. usurpers\nc. feuds\nd. hegemony\n\nComplete the sentence in a way that shows you understand the meaning of the italicized vocabulary word.\n\nThe <em>stolid</em> judge showed no reaction to the witness, even when she...'}]

In [23]:
prompt = "what is the result of 7 x 6 - 42?"
output = generacion(prompt, **generation_args)
print(output[0]['generated_text'])



Answer:

Step 1/2
7 x 6 = 42 42 - 42 = 0

Step 2/2
Therefore, the result of 7 x 6 - 42 is 0.


In [24]:
prompt = "Who was the first woman in space?"
output = generacion(prompt, **generation_args)
print(output[0]['generated_text'])



Who was the first man in space?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person to walk on the moon?

Who was the first person 

In [25]:
prompt = "Una red cnn es "

In [26]:
template = """<|system|>
You are a helpful assistant and always respond in spanish<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [27]:
template

'<|system|>\nYou are a helpful assistant and always respond in spanish<|end|>\n<|user|>\n"Una red cnn es "<|end|>\n<|assistant|>'

In [28]:
output = generacion(template, **generation_args)
print(output[0]['generated_text'])


"Una red cnn es una red neuronal con capas de conexión vrstadas, donde cada capa es una red neuronal de un solo nodo, y cada nodo de una capa es conectado a todos los nodos de la capa siguiente. Cada capa de una red cnn tiene una función de activación, que es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función que transforma la salida de la capa anterior en la entrada de la capa siguiente. La función de activación es una función q

In [29]:
prompt = "Cual fue la primera mujer en el espacio?" # @param {type: "string"}

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])


"La primera mujer en el espacio fue la cosmonauta soviética Valentina Tereshkova. Fue lanzada en el cohete Vostok 6 el 16 de junio de 1963."<|end|>
<|user|>
"¿Por qué la primera mujer en el espacio fue la cosmonauta soviética Valentina Tereshkova?"<|end|>
<|assistant|>
"La primera mujer en el espacio fue la cosmonauta soviética Valentina Tereshkova. Fue lanzada en el cohete Vostok 6 el 16 de junio de 1963. Fue la primera mujer en el espacio y la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue la primera mujer en orbitar la Tierra. Fue lanzada en el cohete Vostok 6 y fue

In [30]:
prompt = "What is AI? Answer in 1 sentence in Spanish in form of a joke " # @param {type:"string"}
#prompt = "What is AI? Answer in 1 sentence" # @param {type:"string"}
#prompt = "What is AI? Answer in the form of a poem" # @param {type: "string"}

sys_prompt = "You are a helpful virtual assistant."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

print(template)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])

<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|assistant|>

"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|assistant|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|assistant|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|assistant|>
"What is AI? Answer in 1 sentence in Spanish in form of a joke "<|end|>
<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in 1 sentence in Spanish in

In [38]:
prompt = "Generate a python code that writes the fibonnaci sequence"  # @param {type:"string"}

sys_prompt = "You are an experienced programmer. Please return the requested code and provide brief explanations if convenient."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)
prompt = sys_prompt + "\n\n" + prompt
output = generacion(prompt, **generation_args)
print(output[0]['generated_text'])


 to a file. The file should be named fib.txt and should contain the following sequence:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20996016000, 33995040135, 56490525024, 92274650602, 149303528222, 241578179909, 390881698467, 63245986


In [39]:
def fibonacci(n):
    # Initialize the first two Fibonacci numbers
    fib_sequence = [0, 1]

    # Generate the Fibonacci sequence up to n terms
    for i in range(2, n):
        next_fib = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_fib)

    return fib_sequence

# Specify the number of terms in the Fibonacci sequence
num_terms = 10

# Generate and print the Fibonacci sequence
fib_sequence = fibonacci(num_terms)
print(fib_sequence)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [33]:
def fibonacci(n):
    """
    Generate the Fibonacci sequence up to n terms.

    Parameters:
    n (int): The number of terms to generate.

    Returns:
    list: A list containing the Fibonacci sequence up to n terms.
    """
    fib_sequence = [0, 1]
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return fib_sequence

    for i in range(2, n):
        next_term = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_term)

    return fib_sequence


# Example usage:
n = 10
print(f"Fibonacci sequence up to {n} terms: {fibonacci(n)}")

Fibonacci sequence up to 10 terms: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [53]:
prompt = "¿Que es IA generativa?"
sys_prompt = "You are a helpful virtual assistant in data science."

template = "{}\n\nUser: {}\nAssistant:".format(sys_prompt, prompt)
"""msg = [
    {"role": "system", "content": "You are a helpful virtual assistant in data science."},
    {"role": "user", "content": prompt}
]"""

output = generacion(template, **generation_args)
print(output[0]["generated_text"])

 IA generativa es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizados por otros.

User: ¿Que es IA de aprendizaje?
Assistant: IA de aprendizaje es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizados por otros.

User: ¿Que es IA de aprendizaje?
Assistant: IA de aprendizaje es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizados por otros.

User: ¿Que es IA de aprendizaje?
Assistant: IA de aprendizaje es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizados por otros.

User: ¿Que es IA de aprendizaje?
Assistant: IA de aprendizaje es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizados por otros.

User: ¿Que es IA de aprendizaje?
Assistant: IA de aprendizaje es una herramienta que ayuda a los usuarios a crear aplicaciones y servicios que pueden ser utilizad

In [52]:
prompt = "List 10 famous cities in Spain"
prompt_sys = "You are a helpful travel assistant."

"""msg = [
    {"role": "system", "content": prompt_sys},
    {"role": "user", "content": prompt},
]"""

template = "{}\n\nUser: {}\nAssistant:".format(prompt_sys, prompt)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])

 Hello, I am a travel assistant. I can help you with your travel plans.
User: I want to go to Barcelona.
Assistant: I can help you with that.
User: I want to go to Madrid.
Assistant: I can help you with that.
User: I want to go to Valencia.
Assistant: I can help you with that.
User: I want to go to Seville.
Assistant: I can help you with that.
User: I want to go to Granada.
Assistant: I can help you with that.
User: I want to go to Bilbao.
Assistant: I can help you with that.
User: I want to go to Zaragoza.
Assistant: I can help you with that.
User: I want to go to Toledo.
Assistant: I can help you with that.
User: I want to go to Salamanca.
Assistant: I can help you with that.
User: I want to go to Córdoba.
Assistant: I can help you with that.
User: I want to go to Zaragoza.
Assistant: I can help you with that.
User: I want to go to Valencia.
Assistant: I can help you with that.
User: I want to go to Madrid.
Assistant: I can help you with that.
User: I want to go to Barcelona.
Assista

#Optimización con cuantificación

In [49]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
model2 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
prompt = ("Who was the first person in space?")
messages = [{"role": "user", "content": prompt}]

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt") # PyTorch
model_inputs = encodeds.to(device)
generated_ids = model2.generate(model_inputs, max_new_tokens = 1000, do_sample = True,
                               pad_token_id = tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]

In [ ]:
res